# RPDA8412 — GDP Growth Forecasting (ARIMA, Holt-Winters, RF, XGBoost)
This notebook:
1. Loads and reshapes World Bank GDP growth data  
2. Splits by country (train/test)  
3. Fits 4 forecasting models  
4. Evaluates using RMSE, MAE, MAPE  
5. Exports plots & tables for the report


In [2]:
import re
import itertools
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA

RAW_CSV = Path("../Data/WDI_GDP_Growth_Data.csv")   
OUT_DIR = Path("../Reports")
FIG_DIR = OUT_DIR / "Figures"
PROC_DIR = Path("../Data/Processed")


for p in [OUT_DIR, FIG_DIR, PROC_DIR]:
    p.mkdir(parents=True, exist_ok=True)


COUNTRIES = {
    "USA": "United States",
    "DEU": "Germany",
    "CHN": "China",
    "IND": "India",
    "BRA": "Brazil",
    "NGA": "Nigeria",
    "KEN": "Kenya",
    "BGD": "Bangladesh",
}

TEST_YEARS = 8   
MAX_LAG = 3      

plt.rcParams["figure.dpi"] = 120


In [3]:
def _is_year_col(c): 
    return bool(re.search(r"(19|20)\d{2}", str(c)))

def _extract_year(c):
    m = re.search(r"(19|20)\d{2}", str(c))
    return int(m.group()) if m else None

def load_wdi(csv_path, country_filter):
    df = pd.read_csv(csv_path)
    year_cols = [c for c in df.columns if _is_year_col(c)]
    id_cols = [c for c in df.columns if c not in year_cols]
    year_map = {c: _extract_year(c) for c in year_cols}

    # Melt wide → long
    m = df.melt(id_vars=id_cols, value_vars=year_cols, var_name="YearCol", value_name="GDP_Growth")
    m["Year"] = m["YearCol"].map(year_map).astype(int)
    m.drop(columns="YearCol", inplace=True)

    # Standardize column names
    rename = {}
    for col in m.columns:
        low = col.lower()
        if low.startswith("country name"):
            rename[col] = "Country"
        elif low.startswith("country code"):
            rename[col] = "CountryCode"
        elif low.startswith("series name"):
            rename[col] = "SeriesName"
        elif low.startswith("series code"):
            rename[col] = "SeriesCode"
    m.rename(columns=rename, inplace=True)

    # Filter countries + numeric clean
    m["GDP_Growth"] = pd.to_numeric(m["GDP_Growth"], errors="coerce")
    m = m[m["CountryCode"].isin(country_filter.keys())]
    m["Country"] = m["CountryCode"].map(country_filter)

    return m.dropna(subset=["GDP_Growth"]).sort_values(["CountryCode", "Year"]).reset_index(drop=True)

# Load dataset
df = load_wdi(RAW_CSV, COUNTRIES)
df.to_csv(PROC_DIR / "gdp_growth_long.csv", index=False)

df.head()


,Country,CountryCode,SeriesName,SeriesCode,GDP_Growth,Year
0,Bangladesh,BGD,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,5.619852,1970
1,Bangladesh,BGD,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,-5.479483,1971
2,Bangladesh,BGD,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,-13.973729,1972
3,Bangladesh,BGD,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,3.325680,1973
4,Bangladesh,BGD,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,9.591956,1974


In [4]:
print("Year Range:", int(df["Year"].min()), "→", int(df["Year"].max()))
print("\nNumber of records per country:")
display(df.groupby("Country")["GDP_Growth"].count())

print("\nBasic Statistics by Country:")
display(df.groupby("Country")["GDP_Growth"].describe()[["count","mean","std","min","max"]])


Year Range: 1970 → 2023

Number of records per country:


Country
Bangladesh       54
Brazil           54
China            54
Germany          54
India            54
Kenya            54
Nigeria          54
United States    54
Name: GDP_Growth, dtype: int64


Basic Statistics by Country:


,count,mean,std,min,max
Country,,,,,
Bangladesh,54.0,4.492802,3.580093,-13.973729,9.591956
Brazil,54.0,3.551474,4.128077,-4.350000,13.968722
China,54.0,8.734495,3.615181,-1.570000,19.300000
Germany,54.0,1.912480,2.097482,-5.545165,5.255006
India,54.0,5.440326,3.188396,-5.777725,9.689592
Kenya,54.0,4.445920,4.008581,-4.655447,22.173892
Nigeria,54.0,3.796544,6.165656,-13.127880,25.007242
United States,54.0,2.745891,2.050941,-2.576500,7.236453


In [5]:
def split_last_n(data, n_last):
    data = data.sort_values("Year").reset_index(drop=True)
    return data.iloc[:-n_last], data.iloc[-n_last:]

splits = {}
for code in COUNTRIES:
    d = df[df["CountryCode"] == code]
    train, test = split_last_n(d, TEST_YEARS)
    splits[code] = (train, test)


{k: (len(v[0]), len(v[1])) for k,v in splits.items()}


{'USA': (46, 8),
 'DEU': (46, 8),
 'CHN': (46, 8),
 'IND': (46, 8),
 'BRA': (46, 8),
 'NGA': (46, 8),
 'KEN': (46, 8),
 'BGD': (46, 8)}

In [6]:
import warnings
warnings.filterwarnings("ignore")

# === Classical Models (ARIMA and Holt-Winters) ===
def fit_arima_grid(train_df, p_vals=(0,1,2), d_vals=(0,1), q_vals=(0,1,2)):
    """
    Runs a small ARIMA grid search by AIC.
    """
    y = train_df["GDP_Growth"].astype(float).values
    best_aic = np.inf
    best_model, best_order = None, None

    for p in p_vals:
        for d in d_vals:
            for q in q_vals:
                try:
                    model = ARIMA(y, order=(p,d,q)).fit()
                    if model.aic < best_aic:
                        best_aic, best_model, best_order = model.aic, model, (p,d,q)
                except:
                    continue
    return best_model, best_order

def forecast_arima(model, steps):
    return np.asarray(model.forecast(steps=steps))

def fit_holt_linear(train_df):
    y = train_df["GDP_Growth"].astype(float).values
    model = ExponentialSmoothing(y, trend="add", seasonal=None).fit(optimized=True)
    return model

def forecast_holt(model, steps):
    return np.asarray(model.forecast(steps))


In [7]:

def make_lag_features(df_country, max_lag=3):
    d = df_country.sort_values("Year").reset_index(drop=True)
    for lag in range(1, max_lag + 1):
        d[f"GDP_Growth_lag{lag}"] = d["GDP_Growth"].shift(lag)
    return d

def prepare_supervised(train, test, max_lag=3):
    # Build lagged features
    tr = make_lag_features(train, max_lag=max_lag).dropna().copy()
    X_train = tr[[f"GDP_Growth_lag{i}" for i in range(1, max_lag + 1)]]
    y_train = tr["GDP_Growth"].astype(float).values

    # For test, concat tail of train + test to compute valid lags
    combined = pd.concat([train[["GDP_Growth"]], test[["GDP_Growth"]]], ignore_index=True)
    for lag in range(1, max_lag + 1):
        combined[f"GDP_Growth_lag{lag}"] = combined["GDP_Growth"].shift(lag)
    X_test = combined.iloc[len(train):][[f"GDP_Growth_lag{i}" for i in range(1, max_lag + 1)]]

    return X_train, y_train, X_test

def fit_rf(X_train, y_train):
    rf = RandomForestRegressor(n_estimators=400, max_depth=5, random_state=42)
    rf.fit(X_train, y_train)
    return rf

def fit_xgb(X_train, y_train):
    xg = XGBRegressor(
        n_estimators=800,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        random_state=42,
        objective="reg:squarederror",
    )
    xg.fit(X_train, y_train)
    return xg


In [8]:

def rmse(y_true, y_pred):
    return float(np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2)))

def mae(y_true, y_pred):
    return float(np.mean(np.abs(np.array(y_true) - np.array(y_pred))))

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denom = np.where(y_true == 0, np.nan, np.abs(y_true))
    return float(np.nanmean(np.abs((y_true - y_pred) / denom) * 100))

results, preds = [], []

for code, (train, test) in splits.items():
    name = COUNTRIES[code]
    y_test = test["GDP_Growth"].astype(float).values


    arima_fit, order = fit_arima_grid(train)
    arima_pred = forecast_arima(arima_fit, len(test))
    results.append({"Country": name, "Model": f"ARIMA{order}",
                    "RMSE": rmse(y_test, arima_pred),
                    "MAE": mae(y_test, arima_pred),
                    "MAPE": mape(y_test, arima_pred)})
    preds.append(pd.DataFrame({"Country": name, "Year": test["Year"], "Model": f"ARIMA{order}",
                               "Actual": y_test, "Pred": arima_pred}))

    
    holt_fit = fit_holt_linear(train)
    holt_pred = forecast_holt(holt_fit, len(test))
    results.append({"Country": name, "Model": "Holt",
                    "RMSE": rmse(y_test, holt_pred),
                    "MAE": mae(y_test, holt_pred),
                    "MAPE": mape(y_test, holt_pred)})
    preds.append(pd.DataFrame({"Country": name, "Year": test["Year"], "Model": "Holt",
                               "Actual": y_test, "Pred": holt_pred}))

    
    X_train, y_train, X_test = prepare_supervised(train, test, MAX_LAG)
    rf = fit_rf(X_train, y_train)
    rf_pred = rf.predict(X_test)
    results.append({"Country": name, "Model": "RandomForest",
                    "RMSE": rmse(y_test, rf_pred),
                    "MAE": mae(y_test, rf_pred),
                    "MAPE": mape(y_test, rf_pred)})
    preds.append(pd.DataFrame({"Country": name, "Year": test["Year"], "Model": "RandomForest",
                               "Actual": y_test, "Pred": rf_pred}))

    
    xg = fit_xgb(X_train, y_train)
    xgb_pred = xg.predict(X_test)
    results.append({"Country": name, "Model": "XGBoost",
                    "RMSE": rmse(y_test, xgb_pred),
                    "MAE": mae(y_test, xgb_pred),
                    "MAPE": mape(y_test, xgb_pred)})
    preds.append(pd.DataFrame({"Country": name, "Year": test["Year"], "Model": "XGBoost",
                               "Actual": y_test, "Pred": xgb_pred}))


results_df = pd.DataFrame(results).sort_values(["Country", "RMSE"])
preds_df = pd.concat(preds, ignore_index=True)

results_df.to_csv(OUT_DIR / "model_results.csv", index=False)
preds_df.to_csv(OUT_DIR / "all_predictions.csv", index=False)

display(results_df)


,Country,Model,RMSE,MAE,MAPE
28,Bangladesh,"ARIMA(2, 1, 1)",1.283902,1.078432,20.232872
30,Bangladesh,RandomForest,1.661149,1.510878,24.386120
29,Bangladesh,Holt,1.760931,1.159709,25.156668
31,Bangladesh,XGBoost,2.057336,1.717974,24.965175
19,Brazil,XGBoost,2.990381,1.980324,73.427552
16,Brazil,"ARIMA(0, 1, 1)",2.994481,2.780133,102.524557
18,Brazil,RandomForest,3.195395,2.300807,81.778028
17,Brazil,Holt,3.694046,3.372150,127.109397
10,China,RandomForest,2.298468,1.762216,47.732415
11,China,XGBoost,2.976106,2.210959,58.064848


In [9]:
def plot_actual_vs_pred(df_plot, title, out_path):
    plt.figure()
    plt.plot(df_plot["Year"], df_plot["Actual"], label="Actual", marker="o")
    plt.plot(df_plot["Year"], df_plot["Pred"], label="Pred", marker="x")
    plt.title(title)
    plt.xlabel("Year")
    plt.ylabel("GDP Growth (annual %)")
    plt.legend()
    plt.tight_layout()
    out_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_path, dpi=150)
    plt.close()


for country in preds_df["Country"].unique():
    df_country = preds_df[preds_df["Country"] == country]
    for model in df_country["Model"].unique():
        d = df_country[df_country["Model"] == model]
        file_path = FIG_DIR / f"{country}_{model.replace(' ', '_')}.png"
        plot_actual_vs_pred(d, f"{country} — {model}", file_path)

print("✅ All plots saved to:", FIG_DIR)


✅ All plots saved to: ..\Reports\Figures


In [10]:

avg_metrics = results_df.groupby("Model")[["RMSE","MAE","MAPE"]].mean().round(3)
display(avg_metrics)


best_models = results_df.loc[results_df.groupby("Country")["RMSE"].idxmin(), ["Country","Model","RMSE"]]
display(best_models)


,RMSE,MAE,MAPE
Model,,,
"ARIMA(0, 0, 1)",2.136,1.292,47.644
"ARIMA(0, 1, 1)",3.835,2.978,116.277
"ARIMA(0, 1, 2)",2.328,1.620,140.951
"ARIMA(1, 1, 1)",3.990,3.415,90.414
"ARIMA(2, 1, 1)",1.750,1.325,127.588
Holt,3.144,2.369,112.046
RandomForest,3.075,2.296,109.245
XGBoost,3.450,2.556,119.470


,Country,Model,RMSE
28,Bangladesh,"ARIMA(2, 1, 1)",1.283902
19,Brazil,XGBoost,2.990381
10,China,RandomForest,2.298468
5,Germany,Holt,2.202973
12,India,"ARIMA(0, 1, 1)",4.759603
24,Kenya,"ARIMA(2, 1, 1)",2.216554
21,Nigeria,Holt,2.955209
1,United States,Holt,2.119676
